In [31]:
import sys
sys.path.append('../../../')
from sklearn.datasets import *
from data_tools import *
from data_tools import PandasTools as pt
from data_tools import ChartTools as ct
from data_tools import AnalyzeTools as at
from data_tools import SKLTools as st
from sklearn.preprocessing import LabelEncoder
ct.IPYTHON_MODE = True
import pandas as pd

train = pt.read_csv("train.csv")
target = train['SalePrice'].astype(float)

data = pt.read_csv("data_feature_engineering.csv")
data.set_index(['Id'], inplace=True)
data = data.astype(float)
data["TotalHouse"] = data["TotalBsmtSF"] + data["1stFlrSF"] + data["2ndFlrSF"]
data["TotalArea"] = data["TotalBsmtSF"] + \
data["1stFlrSF"] + data["2ndFlrSF"] + data["GarageArea"]
data["TotalHouse"] = data["TotalBsmtSF"] + data["1stFlrSF"] + data["2ndFlrSF"]   
data["TotalArea"] = data["TotalBsmtSF"] + data["1stFlrSF"] + data["2ndFlrSF"] + data["GarageArea"]

data["+_TotalHouse_OverallQual"] = data["TotalHouse"] * data["OverallQual"]
data["+_GrLivArea_OverallQual"] = data["GrLivArea"] * data["OverallQual"]
data["+_oMSZoning_TotalHouse"] = data["oMSZoning"] * data["TotalHouse"]
data["+_oMSZoning_OverallQual"] = data["oMSZoning"] + data["OverallQual"]
data["+_oMSZoning_YearBuilt"] = data["oMSZoning"] + data["YearBuilt"]
data["+_oNeighborhood_TotalHouse"] = data["oNeighborhood"] * data["TotalHouse"]
data["+_oNeighborhood_OverallQual"] = data["oNeighborhood"] + data["OverallQual"]
data["+_oNeighborhood_YearBuilt"] = data["oNeighborhood"] + data["YearBuilt"]
data["+_BsmtFinSF1_OverallQual"] = data["BsmtFinSF1"] * data["OverallQual"]

data["-_oFunctional_TotalHouse"] = data["oFunctional"] * data["TotalHouse"]
data["-_oFunctional_OverallQual"] = data["oFunctional"] + data["OverallQual"]
data["-_LotArea_OverallQual"] = data["LotArea"] * data["OverallQual"]
data["-_TotalHouse_LotArea"] = data["TotalHouse"] + data["LotArea"]
data["-_oCondition1_TotalHouse"] = data["oCondition1"] * data["TotalHouse"]
data["-_oCondition1_OverallQual"] = data["oCondition1"] + data["OverallQual"]


data["Bsmt"] = data["BsmtFinSF1"] + data["BsmtFinSF2"] + data["BsmtUnfSF"]
data["Rooms"] = data["FullBath"]+data["TotRmsAbvGrd"]
data["PorchArea"] = data["OpenPorchSF"]+data["EnclosedPorch"]+data["3SsnPorch"]+data["ScreenPorch"]
data["TotalPlace"] = data["TotalBsmtSF"] + data["1stFlrSF"] + data["2ndFlrSF"] + data["GarageArea"] + data["OpenPorchSF"]+data["EnclosedPorch"]+data["3SsnPorch"]+data["ScreenPorch"]



train_data = data[:1460]
test_data = data[1460:]

============= Schema Info ===============
Path:train.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl

### 特征选择

In [32]:
# 特征选择
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel

# 通过模型做特征选择
# reg = RandomForestRegressor(n_estimators=800)
# reg.fit(train_data, target)
# at.show_feature_importance(train_data.columns, reg.feature_importances_)

# selected_cols = st.select_features_from_model(reg, train_data, target)
# train_data_new = train_data[selected_cols]
# test_data_new = test_data[selected_cols]

# 降噪、PCA
scaler = RobustScaler()
train_data_scaled = scaler.fit_transform(train_data)
target_log = np.log(target)
test_data_scaled = scaler.transform(test_data)

# pca = PCA(n_components=355)
# train_data_scaled=pca.fit_transform(train_data_scaled)
# test_data_scaled = pca.transform(test_data_scaled)

### 模型选择

In [33]:
from xgboost import XGBRegressor
from sklearn.ensemble import *
from sklearn.linear_model import *
from sklearn.svm import SVR, LinearSVR
from sklearn.kernel_ridge import KernelRidge
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score  # , GridSearchCV, KFold


def rmse_cv(model, X, y):
    rmse = np.sqrt(-cross_val_score(model, X, y,
                                    scoring="neg_mean_squared_error", cv=5))
    return rmse


models = [Ridge(), Lasso(alpha=0.01, max_iter=10000), RandomForestRegressor(), GradientBoostingRegressor(), SVR(), LinearSVR(),
          ElasticNet(alpha=0.001, max_iter=10000), SGDRegressor(
              max_iter=1000, tol=1e-3), BayesianRidge(), KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5),
          ExtraTreesRegressor(), XGBRegressor(), LGBMRegressor()]

names = ["Ridge", "Lasso", "RF", "GBR", "SVR", "LinSVR",
         "Ela", "SGD", "Bay", "Ker", "Extra", "Xgb", "LGB"]
for name, model in zip(names, models):
    score = rmse_cv(model, train_data_scaled, target_log)
    print("{}: {:.6f}, {:.4f}".format(name, score.mean(), score.std()))

Ridge: 0.135139, 0.0207
Lasso: 0.140528, 0.0269
RF: 0.137459, 0.0099
GBR: 0.124604, 0.0096
SVR: 0.135097, 0.0159
LinSVR: 0.157116, 0.0350
Ela: 0.126118, 0.0191
SGD: 209.556085, 369.8627
Bay: 0.126680, 0.0188
Ker: 0.148907, 0.0606
Extra: 0.135856, 0.0112
Xgb: 0.126845, 0.0101
LGB: 0.125499, 0.0094


In [ ]:
# LR: 2327525147.563259, 3506171844.0878
# Ridge: 0.117521, 0.0091
# Lasso: 0.121117, 0.0061
# RF: 0.139075, 0.0075
# GBR: 0.123911, 0.0077
# SVR: 0.112749, 0.0048
# LinSVR: 0.121610, 0.0085
# Ela: 0.111394, 0.0060
# SGD: 0.163786, 0.0152
# Bay: 0.110557, 0.0060
# Ker: 0.109235, 0.0055
# Extra: 0.135849, 0.0059
# Xgb: 0.126093, 0.0067
# LGB: 0.129974, 0.0066

### 模型调优

#### SVR、LGB、Ela、Ker

In [35]:
from sklearn.model_selection import GridSearchCV


def grid_search(model, X, y, param_grid):
    grid_search = GridSearchCV(
        model, param_grid, cv=5, scoring="neg_mean_squared_error")
    grid_search.fit(X, y)
    print(grid_search.best_params_, np.sqrt(-grid_search.best_score_))
    print(grid_search.best_estimator_)
#     grid_search.cv_results_[
#         'mean_test_score'] = np.sqrt(-grid_search.cv_results_['mean_test_score'])
#     print(pd.DataFrame(grid_search.cv_results_)[
#           ['params', 'mean_test_score', 'std_test_score']])


#"categorical_feature": ",".join( map(lambda x:str(x),(range(0,len(dummies_list))))),
# param_grid = {'n_estimators': [800, 1000, 1200],
#               'num_leaves': [15, 32, 35, 40]}
# reg_lgb = lgb.LGBMRegressor()
# grid_search = GridSearchCV(reg_lgb, param_grid, cv=5,
#                            scoring='neg_mean_squared_error')
# grid_search.fit(train_data, target_log)

In [41]:
param_grid={'alpha':[0.2,0.3,0.4], 'kernel':["polynomial"], 'degree':[3],'coef0':[0.8,1]}
grid_search(KernelRidge(),train_data_scaled,target_log,param_grid)

{'alpha': 0.4, 'kernel': 'polynomial', 'degree': 3, 'coef0': 0.8} 0.16690373665
KernelRidge(alpha=0.4, coef0=0.8, degree=3, gamma=None, kernel='polynomial',
      kernel_params=None)


In [ ]:
grid_search(SVR(),train_data_scaled,target_log,{'C':[9,11,13],'kernel':["rbf"],"gamma":[0.0004,0.0005],"epsilon":[0.009,0.001]})

In [ ]:
grid_search(XGBRegressor(),train_data_scaled,target_log,{'n_estimators': [1000, 1200,1500],'num_leaves': [ 32, 35, 40] })

In [36]:
grid_search(LGBMRegressor(),train_data_scaled,target_log,{'n_estimators': [1000, 1200,1500],'num_leaves': [ 32, 35, 40] })
st.regression_model_estimation

{'num_leaves': 32, 'n_estimators': 1500} 0.127003195199
LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=1500,
       n_jobs=-1, num_leaves=32, objective=None, random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=1)


In [7]:
kr=KernelRidge(alpha=0.4, coef0=1, degree=3, gamma=None, kernel='polynomial',
      kernel_params=None)

lgb = LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=1000,
       n_jobs=-1, num_leaves=40, objective=None, random_state=None,
       reg_alpha=0.05, reg_lambda=0.02, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=1)

ela=ElasticNet(alpha=0.005, copy_X=True, fit_intercept=True, l1_ratio=0.1,
      max_iter=10000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

svr=SVR(C=11, cache_size=200, coef0=0.0, degree=3, epsilon=0.009, gamma=0.0004,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)



In [9]:
stack_model = st.StackingBagging(mod=[lgb,ela,svr],meta_model=lgb)

score = rmse_cv(stack_model,train_data_scaled,target_log)
print(score.mean())

0.144381499365


In [10]:
stack_model.fit(train_data_scaled, target_log)
predict =np.exp( stack_model.predict(test_data_scaled))
test_data['SalePrice'] = predict
pt.write_csv(test_data[['SalePrice']].reset_index(), 'submission.csv')

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

